This will be the main notebook, where data is loaded, investigated, prepared, and models are build, trained and experimented with.

In [1]:
import os
import pandas as pd
from torch.utils.data import Dataset
import cv2
import numpy as np
import torch

In [2]:
PATH_IMAGES = os.path.join('Kvasir-SEG', 'images')
PATH_LABELS = os.path.join('Kvasir-SEG', 'bbox')

## Data Loading

In [10]:
def load_images(directory_path):
    """ Returns a list of file paths to all .jpg images in the given directory. """
    image_files = []
    for filename in os.listdir(directory_path):
        if filename.endswith('.jpg'):
            file_path = os.path.join(directory_path, filename)
            image_files.append(file_path)
    return image_files

image_paths = load_images(PATH_IMAGES)
len(image_paths)

1000

In [ ]:
def load_labels(label_dir):
    """ 
    Loads all CSV label files from the specified directory. 
    Open each file, and save corners of each bounding box together with id/filename as new row.
    Combine everything into a single DataFrame and return it.
    """
    all_labels = []

    for filename in sorted(os.listdir(label_dir)):
        if filename.endswith(".csv"):
            file_id = os.path.splitext(filename)[0]
            df = pd.read_csv(os.path.join(label_dir, filename))
            df["id"] = file_id
            all_labels.append(df)

    labels_df = pd.concat(all_labels, ignore_index=True)

    return labels_df

labels_df = load_labels(PATH_LABELS)
labels_df.shape

(1071, 6)

Checking, if some images have multiple bounding boxes:

In [11]:
labels_df['id'].value_counts().head()

id
cju3uhb79gcgr0871orbrbi3x    10
cju414lf2l1lt0801rl3hjllj     4
cju32a52lb9rc0799xi40qs00     3
cju0roawvklrq0799vmjorwfv     3
cju43c92lm5cj0755lorsorfg     3
Name: count, dtype: int64

Checking if some images have no bounding box at all:

In [12]:
image_ids = {os.path.splitext(os.path.basename(path))[0] for path in image_paths}
labeled_ids = set(labels_df['id'].unique())
unlabeled_ids = image_ids - labeled_ids
len(unlabeled_ids)  # should be 0 if all images are labeled

0

In [ ]:
#print a single item from labels_df
labels_df.iloc[0]

class_name                        polyp
xmin                                 38
ymin                                  5
xmax                                430
ymax                                338
id            cju0qkwl35piu0993l0dewei2
Name: 0, dtype: object